# Load the model

In this unit we'll look at how to load a model along with its persisted parameter states and inference model predictions.

In [ ]:
%matplotlib inline
import torch
import onnxruntime
from torch import nn
import torch.onnx as onnx
import torchvision.models as models
from torchvision import datasets
from torchvision.transforms import ToTensor

To load the model, we'll define the model class which contains the state and parameters of the neural network used to train the model.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

When loading model weights, we needed to instantiate the model class first, because the class defines the structure of a network. Next, we load the parameters using the `load_state_dict()` method.

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load('data/model.pth'))
model.eval()

> **Note:** Be sure to call the `model.eval()` method before inferencing to set the dropout and batch normalization layers to evaluation mode. Otherwise, you'll see inconsistent inference results.


## Model Inference

Optimizing a models to run on a variety of platforms and programming languages is difficult. It's very time-consuming to maximize performance across all the different combinations of frameworks and hardware. The **Open Neural Network Exchange (ONNX)** runtime provides a solution for you to train once and accelerate inference on any hardware, cloud, or edge devices. 

ONNX is a common format supported by a number of vendors to share neural networks and other machine learning models. You can use ONNX format to do inference on your model on other programming languages and frameworks such as Java, JavaScript, C# and ML.NET.

## Exporting the model to ONNX

PyTorch also has native ONNX export support. Given the dynamic nature of the PyTorch execution graph, however, the export process must traverse the execution graph to produce a persisted ONNX model. For this reason, a test variable of the appropriate size should be passed in to the export routine (in our case, we will create a dummy zero tensor of the correct size.  You can get the size from the `shape` fuction on your training dataset with `tensor.shape`):

In [ ]:
input_image = torch.zeros((1,28,28))
onnx_model = 'data/model.onnx'
onnx.export(model, input_image, onnx_model)

We will use our test dataset as sample data for inference from the ONNX model to make predictions.


In [ ]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
x, y = test_data[0][0], test_data[0][1]

We create an inference session with `onnxruntime.InferenceSession`. To inference the ONNX model, call `run` and pass in the list of outputs you want returned (leave empty if you want all of them) and a map of the input values. The result is a list of the outputs.

In [ ]:
session = onnxruntime.InferenceSession(onnx_model, None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

result = session.run([output_name], {input_name: x.numpy()})
predicted, actual = classes[result[0][0].argmax(0)], classes[y]
print(f'Predicted: "{predicted}", Actual: "{actual}"')

The ONNX model enables you to run inference on different platforms and in different programming languages. 